<a href="https://colab.research.google.com/github/fadman/DataEng24/blob/main/Copy_of_Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit =5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [3]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [8]:
%%capture
pip install dlt

In [18]:
import dlt


generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='homework')


# we can load any generator to a table at the pipeline destination as follows:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708061971.4626117 is LOADED and contains no failed jobs


In [19]:
info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708061975.7970734 is LOADED and contains no failed jobs


In [20]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n http_download table below:")

my_people = conn.sql("SELECT * FROM people").df()
display(my_people)

print (f"Sum of Ages {my_people['age'].sum()}")

# As you can see, the same data was loaded in both cases.

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708061971.4626117,SAiAEF3hA+eW2w,None
1,2,Person_2,27,City_A,1708061971.4626117,igQvkdzD9kfDnw,None
2,3,Person_3,28,City_A,1708061971.4626117,o6CoX5B6fV6PvA,None
3,4,Person_4,29,City_A,1708061971.4626117,UEwh+0G6RRDfrA,None
4,5,Person_5,30,City_A,1708061971.4626117,IGlb+K6Aws/pGA,None
5,3,Person_3,33,City_B,1708061975.7970734,5NsOu6p9rFDhmg,Job_3
6,4,Person_4,34,City_B,1708061975.7970734,7hysmal6lZQoJw,Job_4
7,5,Person_5,35,City_B,1708061975.7970734,YltG/9m9cL1HyQ,Job_5
8,6,Person_6,36,City_B,1708061975.7970734,bSJm5A3tPrOKDg,Job_6
9,7,Person_7,37,City_B,1708061975.7970734,vTuZiBbBOd3atg,Job_7


Sum of Ages 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [11]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [24]:
import dlt


generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='homework')


# we can load any generator to a table at the pipeline destination as follows:
info = generators_pipeline.run(people_1(),
										table_name="people_q4",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708062860.993264 is LOADED and contains no failed jobs


In [26]:
info = generators_pipeline.run(people_2(),
										table_name="people_q4",
										write_disposition="merge",
                    primary_key='id')

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset homework
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708062873.562513 is LOADED and contains no failed jobs


In [27]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n http_download table below:")

my_people = conn.sql("SELECT * FROM people_q4").df()
display(my_people)

print (f"Sum of Ages {my_people['age'].sum()}")

# As you can see, the same data was loaded in both cases.

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_q4           │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708062860.993264,nZHhEv7/xuRSOA,None
1,2,Person_2,27,City_A,1708062860.993264,Qp2hSWxq6fLiuw,None
2,5,Person_5,35,City_B,1708062873.562513,STxk2LhxwXEAew,Job_5
3,7,Person_7,37,City_B,1708062873.562513,KZ5VESQbC/Calw,Job_7
4,8,Person_8,38,City_B,1708062873.562513,+1tB/7Zbm8Z0wQ,Job_8
5,4,Person_4,34,City_B,1708062873.562513,3AcKZLcm5VH8Fw,Job_4
6,3,Person_3,33,City_B,1708062873.562513,LoHTlsnjfu0+Fg,Job_3
7,6,Person_6,36,City_B,1708062873.562513,V7S2Jligoe6CoQ,Job_6


Sum of Ages 266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX